# Paper - A tutorial on traffic forecasting with deep learning: [Tutorial Deep Learning](https://www.researchgate.net/profile/Giovanni-Buroni-2/publication/348930068_A_Tutorial_on_Network-Wide_Multi-Horizon_Traffic_Forecasting_with_Deep_Learning/links/6017c45a92851c2d4d0aa267/A-Tutorial-on-Network-Wide-Multi-Horizon-Traffic-Forecasting-with-Deep-Learning.pdf)


## General Import

In [ ]:
import numpy as np 
import os 
import pandas as pd 
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import tensorflow_probability as tfp
import gc
import time

In [ ]:
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
from geopandas.tools import sjoin
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting
import seaborn as sns; sns.set()
from IPython.display import Image
from branca.colormap import  linear
import json
import branca.colormap as cm
import folium

# Seed

In [ ]:
from numpy.random import seed

# Reproducability
def set_seed(seed=31415):
    
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
set_seed(31415)

# Check Files

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import OBU File

In [ ]:
# BXL_timeseries_kaggle.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
new_table = pd.read_csv('../input/obu-data-preprocessing/Flow_BEL_street_30min.csv')
nRow, nCol = new_table.shape
print(f'There are {nRow} rows and {nCol} columns')

# Visualize Street Network

In [ ]:
df_belgium = gpd.read_file('/kaggle/input/belgium-obu/Belgium_streets.json')

m = folium.Map([50.85045, 4.34878], zoom_start=9, tiles='cartodbpositron')
folium.GeoJson(df_belgium).add_to(m)

m

# Select Streets based on Average Traffic Flow

In [ ]:
mean_value = 10

table_index = new_table.iloc[:,1:]
ALL_STREETS = list(table_index.columns.values)

mean_flow =[]
new_street=[]


for street in ALL_STREETS:
    
    single_street=table_index[street]
    mean = np.mean(single_street)
    mean_flow.append(mean)
    new_street.append(street)
    
df_mean_flow = pd.DataFrame({'street_index':new_street, 'mean_flow': mean_flow})
print('')
print(df_mean_flow.head())
print('')

STREETS = df_mean_flow[(df_mean_flow['mean_flow']>= mean_value)] 
STREETS = STREETS.sort_values(by=['street_index'])
STREETS = list(STREETS.street_index)

print('considering a average traffic flow of ' + str(mean_value)+' per street')
print('')
print('mean traffic flow '+str(mean_value)+ ' ---> number of street segments: ' + str(len(STREETS)))


### Add time-based Covariates

In [ ]:
new_table['Datetime'] = pd.to_datetime(new_table['datetime'])

DATAFRAME = new_table
DATAFRAME = DATAFRAME.drop(['datetime'],axis=1) 
DATAFRAME = DATAFRAME[DATAFRAME.columns.intersection(STREETS)]

# Time-based Covariates

DATAFRAME['minutes'] = new_table['Datetime'].dt.minute
DATAFRAME['hour'] = new_table['Datetime'].dt.hour

DATAFRAME['hour_x']=np.sin(DATAFRAME.hour*(2.*np.pi/23))
DATAFRAME['hour_y']=np.cos(DATAFRAME.hour*(2.*np.pi/23))

DATAFRAME['day'] = new_table['Datetime'].dt.day
DATAFRAME['DayOfWeek'] = new_table['Datetime'].dt.dayofweek

DATAFRAME['WorkingDays'] = DATAFRAME['DayOfWeek'].apply(lambda y: 2 if y < 5 else y)
DATAFRAME['WorkingDays'] = DATAFRAME['WorkingDays'].apply(lambda y: 1 if y == 5 else y)
DATAFRAME['WorkingDays'] = DATAFRAME['WorkingDays'].apply(lambda y: 0 if y == 6 else y)

DATAFRAME = DATAFRAME.drop(['minutes','hour','day'],axis=1)

# temporal features = 4
feat_time = 4

DATAFRAME.head()

# Visualize Traffic Flow at Particular Time

In [ ]:
STREETS = [int(float(s)) for s in STREETS]

df_belgium = df_belgium[df_belgium.index.isin(STREETS)]
df_belgium['Trucks_Flow'] =  DATAFRAME.iloc[2182,:-4].astype(float).values

nbh_count_colormap = linear.YlOrRd_09.scale(0,200)

colormap_dept = cm.StepColormap(
    colors=['#00ae53', '#86dc76', '#daf8aa',
            '#ffe6a4', '#ff9a61', '#ee0028'],
    vmin = 0,
    vmax = 200,
    index=[0, 20, 50, 80, 110, 150, 180])

polygons = df_belgium
m = folium.Map([50.85045, 4.34878], zoom_start= 9, tiles='cartodbpositron')

style_function = lambda x: {
    'fillColor': colormap_dept(x['properties']['Trucks_Flow']),
    'color': colormap_dept(x['properties']['Trucks_Flow']),
    'weight': 1.5,
    'fillOpacity': 1
}
folium.GeoJson(polygons,
    style_function=style_function).add_to(m)


colormap_dept.caption = 'Traffic Flow (N#Trucks/30min) at (not real) 12:00 a.m.'
colormap_dept.add_to(m)

m

# SPLITTING Training/Testing

In [ ]:
Image("/kaggle/input/image-lstm/MATRIX.jpg")

In [ ]:
val_step = 168*2 + 168*2 # 1 WEEK
test_step = 168*2 # 1 WEEK

# ATTENTION: anything you learn and is not known in advance, must be learnt only from training data!
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_aux = MinMaxScaler(feature_range=(0, 1))

# TRAINING --- (scaler/scaler_aux).fit_transform()
# TESTING --- (scaler/scaler_aux).transform()

# TRAINING SET
TRAIN = DATAFRAME[: -val_step]
train_feat = scaler.fit_transform(TRAIN.values[:,:-feat_time])

# VALIDATION SET
VAL = DATAFRAME[-val_step : -test_step]
valid_feat = scaler.transform(VAL.values[:,:-feat_time])

# TESTING SET
TEST = DATAFRAME[-test_step:]
test_feat = scaler.transform(TEST.values[:,:-feat_time])


# AUX are known in advance
AUX = scaler_aux.fit_transform(DATAFRAME.values[:,-feat_time:])
train_aux = AUX[: -val_step]
valid_aux = AUX[-val_step: -test_step]
test_aux = AUX[-test_step:]


# concate final results
train_feat = np.hstack([train_feat, train_aux])
valid_feat = np.hstack([valid_feat, valid_aux])
test_feat = np.hstack([test_feat, test_aux])


In [ ]:
def inverse_transform(forecasts, scaler):
    # invert scaling
    inv_pred = scaler.inverse_transform(forecasts)
    return inv_pred

## Visualize testing set

In [ ]:
nRow, nCol = DATAFRAME.shape

plt.figure(figsize=(10,5))

length = list(range(DATAFRAME.shape[0]))

plt.plot(length[:TRAIN.shape[0]], np.sum(TRAIN.iloc[:,:-feat_time], axis=1))
plt.plot(length[TRAIN.shape[0]:TRAIN.shape[0]+VAL.shape[0]],np.sum(VAL.iloc[:,:-feat_time], axis=1))
plt.plot(length[TRAIN.shape[0]+VAL.shape[0]:TRAIN.shape[0]+VAL.shape[0]+TEST.shape[0]],np.sum(TEST.iloc[:,:-feat_time], axis=1))
plt.legend(['training','validation','testing'], loc='upper left')
plt.show()

print(f'Consider {nRow} instances (rows) and {nCol} streets segments (columns)')
print('')
print('TRAIN SIZE: '+ str(TRAIN.shape))
print('')
print('VAL SIZE: '+ str(VAL.shape))
print('')
print('TEST SIZE: '+ str(TEST.shape))

# LSTM encoder decoder model - Multivariate Multiple-step ahead Prediction Model

# Direct Approach

# Data Preparation

## *{Batch_Sz, Input_Sq, Feature_Sz}*

In [ ]:
Image("/kaggle/input/image-lstm/DATAPREP.png")

In [ ]:
def prep_data(dataframe, INPUT, OUTPUT, AUX, BATCH):
    
    TOTAL = INPUT + OUTPUT
    
    dataset_feat = tf.data.Dataset.from_tensor_slices(dataframe)    
    aux = tf.data.Dataset.from_tensor_slices(dataframe[:,-AUX:])    
    dataset_labels = tf.data.Dataset.from_tensor_slices(dataframe)

    # features - past observations
    feat = dataset_feat.window(INPUT,  shift=1,  stride=1,  drop_remainder=True) 
    feat = feat.flat_map(lambda window: window.batch(INPUT))
    
    # aux - temporal features
    aux = aux.window(OUTPUT,  shift=1,  stride=1,  drop_remainder=True ).skip(INPUT)
    aux = aux.flat_map(lambda window: window.batch(OUTPUT))
    
    # labels - future observations
    label = dataset_labels.window(OUTPUT, shift=1,  stride=1,  drop_remainder=True).skip(INPUT)
    label = label.flat_map(lambda window: window.batch(OUTPUT))
    
    dataset = tf.data.Dataset.zip(((feat, aux), label))
    
    dataset = dataset.batch(BATCH).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset


# Parameters

In [ ]:
n_total_features = len(DATAFRAME.columns) 

size_input = 12
size_forecast = 12
size_total = size_input + size_forecast
size_aux = feat_time

batch_size = 32
batch_train = batch_size
batch_valid = batch_size
batch_test = 1

windowed_train = prep_data(train_feat, size_input, size_forecast, size_aux, batch_train)
windowed_valid = prep_data(valid_feat, size_input, size_forecast, size_aux, batch_valid)
windowed_test = prep_data(test_feat, size_input, size_forecast, size_aux, batch_test)

latent_dim = 25
EPOCHS = 250 #250 # in the paper 250

In [ ]:
# timestamp for predictions
timestamp = new_table['Datetime']
timestamp_pred = timestamp[-test_step+size_input-1:]

# LSTM Encoder Decoder Architecture

In [ ]:
Image("/kaggle/input/image-lstm/ECDEC.jpg")

### Input Encoder

In [ ]:
# the input for encoder
past_inputs = tf.keras.Input(shape=(size_input, n_total_features), name = 'enc_inputs')
past_inputs

### Encoder

In [ ]:
class Encoder(tf.keras.Model):
    
    def __init__(self, enc_units, sz_input, sz_tot,  batch_sz):
        
        super(Encoder, self).__init__()
        
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        
        self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                         return_sequences=False, #turn to False for one layer
                                         return_state=True,
                                          recurrent_initializer='glorot_uniform',
                                          kernel_regularizer=regularizers.l2(0.001),
                                          name='Encoder')

                
    def __call__(self, x):

        output, state_h, state_c = self.lstm(x) # just one layer on the paper 
        
        return output, state_h, state_c


In [ ]:
encoder = Encoder(latent_dim, size_input, n_total_features,  batch_size)
out_enc, h, c = encoder(past_inputs)
out_enc

### Input Decoder

In [ ]:
# the future input for decoder
future_inputs = tf.keras.Input(shape=(size_forecast, size_aux), name='aux_inputs')
future_inputs

### Decoder

In [ ]:
class Decoder(tf.keras.Model):
    
    def __init__(self, dec_units, sz_forecast, sz_aux, tot_feat,  batch_sz):
        
        super(Decoder, self).__init__()
        
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.feat = tot_feat
        self.out = sz_forecast
        self.sz_aux = sz_aux

        
        self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences = True,
                                        recurrent_initializer='glorot_uniform',
                                         kernel_regularizer=regularizers.l2(0.001),
                                         name ='Decoder') # 
        
        self.dense1 = tf.keras.layers.Dense(self.dec_units, kernel_regularizer=regularizers.l2(0.001))
        
        self.drop = tf.keras.layers.Dropout(0.1)
        
        self.fc = tf.keras.layers.Dense(self.feat, kernel_regularizer=regularizers.l2(0.001)) #
        

    def __call__(self, x, h, c,  training=False):
    
        output_dec = self.lstm(x, initial_state= [h, c]) # just one layer on the paper 
        
        if training:
            
            output_dec = self.drop(output_dec, training =training)

        out = self.fc(output_dec)

        return out
        

        

### Decoder Output

In [ ]:
decoder = Decoder(latent_dim, 12, size_aux, n_total_features,  batch_size)
out_dec = decoder(future_inputs, out_enc, c)
out_dec 

# Train Model

### Optimizer

In [ ]:
opt = tf.keras.optimizers.Adam()

### Mean Absolute Loss function 

In [ ]:
loss_fct = tf.keras.losses.MeanAbsoluteError(name='loss_function')
valid_loss_fct = tf.keras.losses.MeanAbsoluteError(name='valid_loss_function')

### Batch Loss

In [ ]:
@tf.function
def batch_loss(inp, aux, targ, loss_funct, opt=None):
    loss = 0
    with tf.GradientTape() as tape:
        context_vector, state_h, state_c = encoder(inp)
        predictions = decoder(aux, state_h, state_c, training=True)
        loss = loss_funct(targ, predictions)
    if opt is not None:
        variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, variables)
        opt.apply_gradients(zip(gradients, variables))
    return loss

### early stopping

In [ ]:
def monitor_loss(val, epoch, delta, cnt):
    if ((epoch > 0) and (val[epoch-1] - val[epoch])) > delta:
        cnt = 0    
    else:
        cnt += 1 
        
    return cnt

### Training and Validation

In [ ]:
print('')
print('Training & Validation')
print('')

# early stopping
patience = 20   
min_delta = 0.0001   
patience_cnt = 0 

# Keep results for plotting
train_loss_results = []
valid_loss_results = []
steps_per_epoch = len(TRAIN) // batch_size

start = time.time()

for epoch in range(EPOCHS):
    
    ## training
    step = 0
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    for (batch, (inp_tot, targ)) in enumerate(windowed_train.take(steps_per_epoch)):
        
        inp = inp_tot[0]
        aux = inp_tot[1]
        
        batch_loss_results = batch_loss(inp, aux, targ, loss_fct, opt)
        
        # training progress
        epoch_loss_avg.update_state(batch_loss_results)

    # collect training loss values
    train_loss_results.append(epoch_loss_avg.result())
    
    ## validation
    step = 0
    epoch_valid_loss_avg = tf.keras.metrics.Mean()
    
    for (batch, (inp_tot, targ)) in enumerate(windowed_valid.take(steps_per_epoch)):
        
        inp = inp_tot[0]
        aux = inp_tot[1]
        
        batch_loss_results = batch_loss(inp, aux, targ, valid_loss_fct, None)
        
        # training progress
        epoch_valid_loss_avg.update_state(batch_loss_results)

    # collect training loss values
    valid_loss_results.append(epoch_valid_loss_avg.result())
    
    if epoch % 10 == 0:
        
        print("Epoch {}: Loss MAE: {:.5f} --- Val Loss MAE: {:.5f}".format(epoch,
                                                                       epoch_loss_avg.result(),
                                                                       epoch_valid_loss_avg.result()))
        
      # ----- EARLY STOPPING -------
    
    patience_cnt = monitor_loss(valid_loss_results, epoch, min_delta, patience_cnt)

    if patience_cnt > patience:
        
        print("early stopping...") 
        break  
        
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# Plot Training Progress

In [ ]:
fig, axes = plt.subplots(1, sharex=True, figsize=(12, 8))

fig.suptitle('Training Metrics')
axes.set_ylabel("Loss (MAE) - training and validation", fontsize=14)
axes.plot(train_loss_results)
axes.plot(valid_loss_results)
axes.set_xlabel("Epoch", fontsize=14)
plt.show()

In [ ]:
def evaluate_forecasts(targets, forecasts, n_seq):
    
    list_rmse = []
    list_mae = []
    
    for i in range(n_seq):
        true = np.vstack([target[i] for target in targets])
        predicted = np.vstack([forecast[i] for forecast in forecasts])
        
        rmse = np.sqrt((np.square(true - predicted)).mean(axis=0))
        mae = np.absolute(true - predicted).mean(axis=0)
        
        list_rmse.append(rmse)
        list_mae.append(mae)
        
    list_rmse = np.vstack(list_rmse)
    list_mae = np.vstack(list_mae)
    
    return list_rmse, list_mae

# Test and Update Model

In [ ]:
forecasts = []
targets = []

rmse_list = []
mae_list = []

print('Starting')
print('')

for (step, (inp_tot, targ)) in enumerate(windowed_test):
           

        inp = inp_tot[0]
        aux = inp_tot[1]
        
        targ = tf.cast(targ, tf.float32)
        
        out_enc, state_h, state_c  = encoder(inp)
        
        pred = decoder(aux, state_h, state_c, training=False)
        
        truth = inverse_transform(targ[0][:,:-size_aux],  scaler)
        pred = inverse_transform(pred[0][:,:-size_aux],  scaler)
        
        forecasts.append(pred)
        targets.append(truth)
        
        rmse, mae = evaluate_forecasts(targets, forecasts, size_forecast)
           
        rmse_list.append(rmse)
        mae_list.append(mae)
        
        
        print('* Time step '+str(step))
        print('* Timestamp '+str(timestamp_pred.iloc[step]))
        print('* Prediction Accuracy (MAE) '+ str(np.absolute(truth - pred).mean()))
        print('* After prediction UPDATE model with new streets observations')
        
        new_instance = test_feat[step,:].reshape(1,-1)
    
        train_feat = np.vstack([train_feat[1:,:], new_instance])
    
        windowed_new = prep_data(train_feat, size_input, size_forecast, size_aux, batch_size) 

        update_steps_per_epoch = len(train_feat)//batch_size
        
        UPDATE = 2
        
        for epoch in range(UPDATE):
            
            for (batch, (inp_tot_new, targ_new)) in enumerate(windowed_new.take(update_steps_per_epoch )):
                
                inp_new = inp_tot_new[0]
                aux_new = inp_tot_new[1]
                
                targ_new = tf.cast(targ, tf.float32)
                
                batch_loss_results = batch_loss(inp_new, aux_new, targ_new, loss_fct, opt)
                
                # Track progress
                epoch_loss_avg.update_state(batch_loss_results)
                
            # End epoch
            train_loss_results.append(epoch_loss_avg.result())
            
            if epoch % UPDATE == 0:
                print("UPDATE - Epoch {}: Loss MAE: {:.3f}".format(epoch, epoch_loss_avg.result()))
        
        # ---> comment this to have full prediction
        if step == 5:
            break
            
        print('')     

# Performance Metrics for each forecasting horizon (here for first 5 predictions steps).

To get metrics for the full test set, comment the above

```
if step == 5:
            break
```

# RMSE Metric

In [ ]:
RMSE_MEAN = np.mean(rmse_list,axis=0).mean(axis=1)
RMSE_STD =  np.std(rmse_list,axis=0).std(axis=1)

for i in range(len(RMSE_MEAN)):
    print('t+'+str(i+1)+' RMSE MEAN ' +str(np.round(RMSE_MEAN[i],3))+' +- '+str(np.round(RMSE_STD[i],3)))
    print('')

# MAE Metric

In [ ]:
MAE_MEAN = np.mean(mae_list,axis=0).mean(axis=1)
MAE_STD =  np.std(mae_list,axis=0).std(axis=1)

for i in range(len(MAE_MEAN)):
    print('t+'+str(i+1)+' MAE MEAN ' +str(np.round(MAE_MEAN[i],3))+' +- '+str(np.round(MAE_STD[i],3)))
    print('')

# Example Visualizations Results for Time step t

#### Matplotlib

In [ ]:
t = timestamp_pred.iloc[step+1:step+13]
        
       
plt.fill_between(t,  np.mean(pred, axis=1) + np.std(pred, axis=1), 
           (np.mean(pred, axis=1) - np.std(pred, axis=1)),
           color = 'green', label = 'pred mean +- std', alpha=0.13,
           linewidth = 2)

plt.fill_between(t,  np.mean(truth, axis=1) + np.std(truth, axis=1), 
           (np.mean(truth, axis=1) - np.std(truth, axis=1)),
           color = 'orange', label = 'targ mean +- std', alpha=0.13,
           linewidth = 2)

plt.plot(t, np.mean(pred, axis=1), color = 'green', lw=2, label='Mean Prediction') 
plt.plot(t, np.mean(truth, axis=1), color = 'orange', lw=2, label='Mean Truth')

plt.title('$Mean\ Traffic\ Flow\ in\ Belgium$ at: '+str(timestamp_pred.iloc[step]))
plt.ylabel('$Traffic\ Flow$')
plt.xlabel('$Forecasting\ Horizon$')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.show()

### pydeck

In [ ]:
!pip install pydeck

#### visualization for prediction at (t+1)

In [ ]:
import pydeck as pdk

df_belgium['traffic_flow'] = pred[0]
print('Prediction for time: ' +str(timestamp_pred.iloc[step+1]))

tooltip = {"text": "Traffic Flow : {traffic_flow}"}

INITIAL_VIEW_STATE = pdk.ViewState(latitude=50.85045, longitude=4.34878, zoom=8, max_zoom=30, pitch=10, bearing=0)
geojson = pdk.Layer(
"GeoJsonLayer",
df_belgium,
stroked=False,
filled=True,
extruded=True,
wireframe=True,
get_elevation = "traffic_flow*5",
get_fill_color='[255, (1-traffic_flow/250)*255, 0]',
get_line_color='[255, 255, 255]')

r = pdk.Deck(map_style=pdk.map_styles.LIGHT, layers=geojson, initial_view_state=INITIAL_VIEW_STATE,tooltip={"text": "Traffic Flow: {traffic_flow}"},)
r.to_html("geojson_layer.html", notebook_display=True)




# Save Results Pickle

In [ ]:
import pickle

# Saving the objects:
with open('save_predictions_results.pkl', 'wb') as f: 
    pickle.dump([rmse_list, mae_list], f)